In [2]:
from bigbrotr import Bigbrotr
from event import Event
from relay import Relay
from relay_metadata import RelayMetadata
import utils

In [9]:
import sys
import json
import time
import websocket
import ssl
import socks  # Provided by PySocks
import socket

def fetch_nostr_events(url, network, start, stop):
    # Generate a random subscription ID
    events = []
    sub_id = "" + str(int(time.time()))

    # Filter to get all events between start and stop
    req = [
        "REQ",
        sub_id,
        {
            "since": int(start),
            "until": int(stop)
        }
    ]

    close_req = ["CLOSE", sub_id]

    # Configure proxy if using Tor
    if network.lower() == "tor":
        socks.set_default_proxy(socks.SOCKS5, "127.0.0.1", 9050)
        socket.socket = socks.socksocket
        sslopt = {"cert_reqs": ssl.CERT_NONE}
    else:
        sslopt = {"cert_reqs": ssl.CERT_REQUIRED}

    def on_message(ws, message):
        data = json.loads(message)
        if data[0] == "EVENT":
            try:
                Event.from_dict(data[2])
            except Exception as e:
                print(e, data[2])
            events.append(data[2])
        elif data[0] == "EOSE":
            print("End of stored events.")
            ws.send(json.dumps(close_req))
            ws.close()

    def on_error(ws, error):
        print("Error:", error)

    def on_close(ws, close_status_code, close_msg):
        print("Connection closed.")


    def on_open(ws):
        ws.send(json.dumps(req))

    print(f"Connecting to {url} over {network}...")
    ws = websocket.WebSocketApp(
        url,
        on_message=on_message,
        on_error=on_error,
        on_close=on_close,
        on_open=on_open
    )

    ws.run_forever(sslopt=sslopt)
    print("Connection closed.")
    print(f"Fetched {len(events)} events.")
    return events

# Example usage:
events = fetch_nostr_events("wss://filter.nostr.wine", "clearnet", 1747296164-100, 1747296164)
events

Connecting to wss://filter.nostr.wine over clearnet...
Error: 
Connection closed.
Connection closed.
Fetched 0 events.


[]

In [6]:
e = {'id': '05863177df7b8764ee374a0c9f3cafb2ae6ca96dd0f10701d90a1f7f66ae6a19', 'kind': 6, 'pubkey': '1ac0c5ab27cf0468f535805e03578fcfa52d839f8909578776a0391d77ca82f9', 'created_at': 1746924449, 'content': '{"id":"b4d4e1de25919c3ce4f9ab6f60653cb9d819bc001c04a43d359f59bc2de2db6f","pubkey":"b1e1185884a6d14bbfce3899cb53e8183adde642f264d0ff4f1745371e06134c","created_at":1746921626,"kind":1,"tags":[["imeta","url https://blossom.primal.net/dc075c32768e15be88d1d7dc300f0a5c940ca0530bd61a6a646c7ae7e506c925.jpg","m image/jpeg","ox dc075c32768e15be88d1d7dc300f0a5c940ca0530bd61a6a646c7ae7e506c925","dim 1440x1900"]],"content":"The opportunity to own a home miner is now better than ever. 👇👇👇\\nhttps://www.solosatoshi.com/product/bitaxe-gamma/\\n\\nhttps://blossom.primal.net/dc075c32768e15be88d1d7dc300f0a5c940ca0530bd61a6a646c7ae7e506c925.jpg","sig":"d74e1198e693355f27462a546dfe0317168682b39e6cdc89f492d3ea3b95b402a8a04a5351e36ccc88c0f969a88c07c163528910b7936c4d7be4f2216d25bc2e"}', 'tags': [['e', 'b4d4e1de25919c3ce4f9ab6f60653cb9d819bc001c04a43d359f59bc2de2db6f'], ['p', 'b1e1185884a6d14bbfce3899cb53e8183adde642f264d0ff4f1745371e06134c']], 'sig': '43faea4b89464314c14135b0f09f8cbc2e3fd3858a8644d2b153548ac246005050d119e2249cd224bbc2def8876c038726e35db3dfd3d00a04a457341cefcb2f'}
Event.from_dict(e)

Event(id=05863177df7b8764ee374a0c9f3cafb2ae6ca96dd0f10701d90a1f7f66ae6a19, pubkey=1ac0c5ab27cf0468f535805e03578fcfa52d839f8909578776a0391d77ca82f9, created_at=1746924449, kind=6, tags=[['e', 'b4d4e1de25919c3ce4f9ab6f60653cb9d819bc001c04a43d359f59bc2de2db6f'], ['p', 'b1e1185884a6d14bbfce3899cb53e8183adde642f264d0ff4f1745371e06134c']], content={"id":"b4d4e1de25919c3ce4f9ab6f60653cb9d819bc001c04a43d359f59bc2de2db6f","pubkey":"b1e1185884a6d14bbfce3899cb53e8183adde642f264d0ff4f1745371e06134c","created_at":1746921626,"kind":1,"tags":[["imeta","url https://blossom.primal.net/dc075c32768e15be88d1d7dc300f0a5c940ca0530bd61a6a646c7ae7e506c925.jpg","m image/jpeg","ox dc075c32768e15be88d1d7dc300f0a5c940ca0530bd61a6a646c7ae7e506c925","dim 1440x1900"]],"content":"The opportunity to own a home miner is now better than ever. 👇👇👇\nhttps://www.solosatoshi.com/product/bitaxe-gamma/\n\nhttps://blossom.primal.net/dc075c32768e15be88d1d7dc300f0a5c940ca0530bd61a6a646c7ae7e506c925.jpg","sig":"d74e1198e693355f27

In [10]:
async def ping_relay(relay_url):
    rtt_ms = None
    try:
        start = time.time()
        async with websockets.connect(relay_url) as ws:
            end = time.time()
            rtt_ms = int((end - start) * 1000)
    except KeyboardInterrupt:
        raise KeyboardInterrupt
    except Exception as e:
        print(f"Error connecting to {relay_url}: {e}")
    return rtt_ms

def fetch_relay_info(url):
    headers = {'Accept': 'application/nostr+json'}
    data = None
    error = None
    relay_url = url[6:] if url.startswith('wss://') else url[5:] if url.startswith('ws://') else url
    try:
        response = requests.get(f"https://{relay_url}", headers=headers, timeout=10)
        if response.status_code == 200:
            data = response.json()
        else:
            error = response.text
    except Exception:
        try:
            response = requests.get(f"http://{relay_url}", headers=headers, timeout=10)
            if response.status_code == 200:
                data = response.json()
            else:
                error = response.text
        except Exception as e:
            error = str(e)
    return url, {'data': data, 'error': error}

rtt_ms = await ping_relay('wss://relay.damus.io')
url, info = fetch_relay_info('wss://relay.damus.io')

Error connecting to wss://relay.damus.io: name 'websockets' is not defined


In [ ]:
relays = pd.read_csv('../data/raw/relays_url.csv')
for url in relays['url']:
    try:
        async with websockets.connect(url) as ws:
            print(f'Connected to {url}')
    except Exception as e:
        print(f'Failed to connect to {url}: {e}')
    finally:
        print('Done')

In [ ]:
RELAY_URL = "wss://relay.nostrdice.com"  # Cambia con l'URL del relay reale
RELAY_URL = "wss://relay.boroghor.com"  # Cambia con l'URL del relay reale
RELAY_URL = "wss://schnorr.me"
RELAY_URL = "wss://mastodon.cloud/api/v1/streaming"
async def fetch_events(from_ts: int, to_ts: int):
    events = []
    subscription_id = str(uuid.uuid4())[:64]
    filter_obj = {
        # "since": from_ts,
        # "until": to_ts,
        # "limit": 10
    }

    async with websockets.connect(RELAY_URL) as ws:
        print(f"Connesso a {RELAY_URL}")

        req_msg = ["REQ", subscription_id, filter_obj]
        await ws.send(json.dumps(req_msg))
        print(f"Inviato: {req_msg}")

        try:
            while True:
                raw_message = await asyncio.wait_for(ws.recv(), timeout=30)
                message = json.loads(raw_message)

                if not isinstance(message, list):
                    print("Messaggio non valido (non è un array JSON)")
                    continue

                msg_type = message[0]

                if msg_type == "EVENT":
                    _, sub_id, event = message
                    if sub_id == subscription_id:
                        # print(f"Ricevuto evento: {event}")
                        events.append(event)

                elif msg_type == "EOSE":
                    _, sub_id = message
                    if sub_id == subscription_id:
                        print("Fine degli eventi storici.")
                        break

                elif msg_type == "NOTICE":
                    _, notice = message
                    print(f"NOTICE dal relay: {notice}")

                elif msg_type == "CLOSED":
                    _, sub_id, reason = message
                    if sub_id == subscription_id:
                        print(f"Subscription chiusa dal relay: {reason}")
                        break

                elif msg_type == "OK":
                    _, event_id, accepted, message_str = message
                    status = "accettato" if accepted else "rifiutato"
                    print(f"Evento {event_id} {status} - {message_str}")

                else:
                    print(f"Messaggio sconosciuto: {message}")

        except asyncio.TimeoutError:
            print("Timeout: nessun messaggio ricevuto per 30 secondi.")
        finally:
            close_msg = ["CLOSE", subscription_id]
            await ws.send(json.dumps(close_msg))
            print("Subscription chiusa.")
    return events

# Esegui in una cella async
async def main():
    to_time = int(time.time())
    from_time = to_time - 600
    return await fetch_events(0, 1727090175)

events = await main()


In [21]:
import asyncio
import websockets
import time
import json
import logging
from event import Event
import utils
import uuid
import statistics

# Configura log
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Inserisci qui l'URL del relay e l'evento da testare (JSON valido)

async def measure_rtt(url: str, attempts: int = 5):
    rtts = []
    for i in range(attempts):
        try:
            start = time.time()
            async with websockets.connect(url, ping_interval=None):
                rtt = (time.time() - start) * 1000
                rtts.append(rtt)
                logging.info(f"🔁 RTT attempt {i+1}/{attempts}: {rtt:.2f} ms")
        except Exception as e:
            logging.warning(f"❌ RTT attempt {i+1} failed: {e}")
    if rtts:
        return {
            "rtt_avg_ms": round(statistics.mean(rtts), 2),
            "rtt_min_ms": round(min(rtts), 2),
            "rtt_max_ms": round(max(rtts), 2),
            "rtt_stddev_ms": round(statistics.stdev(rtts) if len(rtts) > 1 else 0.0, 2),
            "samples": len(rtts)
        }
    else:
        raise ConnectionError("All RTT attempts failed.")
    
async def test_relay(url: str, event: Event):
    try:
        # event = event.to_dict()
        subscription_id = str(uuid.uuid4())[:64]
        rtt_stats = await measure_rtt(url)
        logging.info(f"⏱️ Connecting to {url}...")
        async with websockets.connect(url, ping_interval=None) as ws:

            # 🟢 Test lettura: invia REQ
            read_filter = json.dumps(["REQ", subscription_id, {"limit": 1}])
            await ws.send(read_filter)
            can_read = False
            logging.info("📥 Sent REQ for reading events...")

            try:
                while True:
                    msg = await asyncio.wait_for(ws.recv(), timeout=5)
                    data = json.loads(msg)
                    if data[0] in ["EVENT", "EOSE"]:
                        can_read = True
                        logging.info(f"✅ Received event or EOSE response: {data[0]}")
                        break
            except asyncio.TimeoutError:
                logging.warning("⚠️ No read response received (timeout).")

            # 🔴 Test scrittura: invia EVENT
            event_msg = json.dumps(["EVENT", event])
            await ws.send(event_msg)
            can_write = False
            logging.info("📤 Sent EVENT to test write...")

            try:
                while True:
                    msg = await asyncio.wait_for(ws.recv(), timeout=5)
                    data = json.loads(msg)
                    if data[0] == "OK" and data[1] == event["id"]:
                        can_write = data[2]  # true or false
                        status = "✅ Accepted" if can_write else "❌ Rejected"
                        logging.info(f"{status}: {data[3]}")
                        break
            except asyncio.TimeoutError:
                logging.warning("⚠️ No write response received (timeout).")

            # 🔚 Chiudi la subscription
            await ws.send(json.dumps(["CLOSE", subscription_id]))

            return {
                "relay": url,
                **rtt_stats,
                "can_read": can_read,
                "can_write": can_write
            }

    except Exception as e:
        logging.error(f"❌ Error: {str(e)}")
        return {"relay": url, "error": str(e)}

# Avvia il test
if __name__ == "__main__":
    relay_url = "wss://relay.damus.io"
    sec, pub = utils.generate_nostr_keypair()
    event = utils.generate_event(sec, pub, int(time.time()), 1, [], "Test event content")
    result = await test_relay(relay_url, event)
    print("\n📊 RISULTATO DEL TEST")
    for k, v in result.items():
        print(f"{k}: {v}")


2025-05-13 21:19:54,979 - INFO - 🔁 RTT attempt 1/5: 744.27 ms
2025-05-13 21:19:55,820 - INFO - 🔁 RTT attempt 2/5: 621.95 ms
2025-05-13 21:19:56,297 - INFO - 🔁 RTT attempt 3/5: 270.14 ms
2025-05-13 21:19:56,848 - INFO - 🔁 RTT attempt 4/5: 306.15 ms
2025-05-13 21:19:57,460 - INFO - 🔁 RTT attempt 5/5: 306.25 ms
2025-05-13 21:19:57,768 - INFO - ⏱️ Connecting to wss://relay.damus.io...
2025-05-13 21:19:58,395 - INFO - 📥 Sent REQ for reading events...
2025-05-13 21:19:58,689 - INFO - ✅ Received event or EOSE response: EVENT
2025-05-13 21:19:58,689 - INFO - 📤 Sent EVENT to test write...
2025-05-13 21:19:58,996 - INFO - ✅ Accepted: 



📊 RISULTATO DEL TEST
relay: wss://relay.damus.io
rtt_avg_ms: 449.75
rtt_min_ms: 270.14
rtt_max_ms: 744.27
rtt_stddev_ms: 217.87
samples: 5
can_read: True
can_write: True


In [ ]:
import psycopg2
import psycopg2.extras
import concurrent.futures
from typing import List
import time
from relay_metadata import RelayMetadata
from your_module import foo  # la funzione foo che recupera i metadati del relay

class Bigbrotr:
    def __init__(self, host: str, port: int, user: str, password: str, dbname: str):
        self.host = host
        self.port = port
        self.user = user
        self.password = password
        self.dbname = dbname
        self.conn = None
        self.cur = None

    def connect(self):
        """Connessione al database."""
        self.conn = psycopg2.connect(
            host=self.host,
            port=self.port,
            user=self.user,
            password=self.password,
            dbname=self.dbname
        )
        self.cur = self.conn.cursor()

    def close(self):
        """Chiude la connessione al database."""
        self.cur.close()
        self.conn.close()

    def fetch_relay_urls(self, limit: int, offset: int) -> List[str]:
        """Recupera gli URL dei relay dal database in modo paginato."""
        query = "SELECT url FROM relays LIMIT %s OFFSET %s"
        self.cur.execute(query, (limit, offset))
        rows = self.cur.fetchall()
        return [row[0] for row in rows]

    def insert_relay_metadata_batch(self, relay_metadata_list: List[RelayMetadata]) -> None:
        """Inserisce i metadati dei relay nel database in batch."""
        query = """
        INSERT INTO relay_metadata (
            relay_url, generated_at, connection_success, nip11_success, readable, writable, 
            rtt, name, description, banner, icon, pubkey, contact, supported_nips, software, 
            version, privacy_policy, terms_of_service, limitation, extra_fields
        ) VALUES %s
        ON CONFLICT (relay_url, generated_at) DO NOTHING;
        """
        values = [
            (
                rm.relay_url,
                rm.generated_at,
                rm.connection_success,
                rm.nip11_success,
                rm.readable,
                rm.writable,
                rm.rtt,
                rm.name,
                rm.description,
                rm.banner,
                rm.icon,
                rm.pubkey,
                rm.contact,
                rm.supported_nips,
                rm.software,
                rm.version,
                rm.privacy_policy,
                rm.terms_of_service,
                rm.limitation,
                rm.extra_fields
            )
            for rm in relay_metadata_list
        ]
        # Inserimento in batch
        psycopg2.extras.execute_values(self.cur, query, values)
        self.conn.commit()

def fetch_and_insert_metadata(bigbrotr: Bigbrotr, url: str) -> RelayMetadata:
    """Recupera i metadati del relay tramite la funzione foo."""
    metadata = foo(url)  # funzione che recupera i metadati dal relay
    if metadata:
        return metadata
    return None

def fetch_all_and_process(bigbrotr: Bigbrotr, batch_size: int = 100):
    """Recupera gli URL dei relay e li processa in parallelo."""
    limit = 100  # numero di URL da recuperare per ogni batch
    offset = 0
    all_metadata = []

    while True:
        relay_urls = bigbrotr.fetch_relay_urls(limit, offset)
        if not relay_urls:
            break  # se non ci sono più URL, interrompiamo il ciclo

        # Usa un ThreadPoolExecutor per eseguire fetch_and_insert_metadata in parallelo
        with concurrent.futures.ThreadPoolExecutor() as executor:
            metadata_list = list(executor.map(lambda url: fetch_and_insert_metadata(bigbrotr, url), relay_urls))
        
        # Filtra i metad ati non None
        metadata_list = [md for md in metadata_list if md is not None]
        all_metadata.extend(metadata_list)

        # Quando raggiungiamo il batch_size, inseriamo i metadati nel DB
        if len(all_metadata) >= batch_size:
            bigbrotr.insert_relay_metadata_batch(all_metadata[:batch_size])
            all_metadata = all_metadata[batch_size:]  # Rimuove il batch appena inserito

        # Aggiorna l'offset per la pagina successiva
        offset += limit

    # Se ci sono metadati rimasti che non sono stati inseriti
    if all_metadata:
        bigbrotr.insert_relay_metadata_batch(all_metadata)

def main():
    host = "localhost"
    port = 5432
    user = "admin"
    password = "admin"
    dbname = "bigbrotr"

    bigbrotr = Bigbrotr(host, port, user, password, dbname)
    bigbrotr.connect()

    try:
        # Avvia la raccolta e l'inserimento dei metadati
        fetch_all_and_process(bigbrotr)
    finally:
        bigbrotr.close()

if __name__ == "__main__":
    main()
